# Cleaning data in python
- we start with the same data and perform the same cleaning steps

In [2]:
import pandas as pd

In [3]:
name = [
  'Wardruna',
  'Skinflower',
  'Behemoth',
  'Konvent',
  'Ultha',
  'Chthonic',
  'Jinjer'
]
country = [
  'Norway',
  'Netherlands',
  'Poland',
  'Denmark',
  'Germany',
  'Taiwan',
  'Ukraine'
]
albums = [5, None, 12, 2, 4, 11, 4]
singles_ep = [3, None, 12, 0, 4, 14, 3]
splits = [1, None, 1, 0, 3, 2, None]
formed = [2003, None, 1991, 2015, 2014, 1995, 2009]

#bands = 

#bands <- dplyr::tibble(name, country, albums, singles_ep, splits, formed)
# calculate age and productivity
#bands <- bands %>% 
#  mutate(age = 2023 - formed) %>% 
#  mutate(productivity = albums / age)

In [4]:
bands = pd.DataFrame(list(zip(
    name, country, albums, singles_ep, splits, formed
)), columns = ['name', 'country', 'albums', 'singles_ep', 'splits', 'formed'])


In [61]:
# calculate productivity
bands['age'] = 2023 - bands['formed']
bands['productivity'] = bands['albums'] / bands['age']

In [62]:
bands.head()

,name,country,albums,singles_ep,splits,formed,age,productivity
0,Wardruna,Norway,5.0,3.0,1.0,2003.0,20.0,0.250000
1,Skinflower,Netherlands,NaN,NaN,NaN,NaN,NaN,NaN
2,Behemoth,Poland,12.0,12.0,1.0,1991.0,32.0,0.375000
3,Konvent,Denmark,2.0,0.0,0.0,2015.0,8.0,0.250000
4,Ultha,Germany,4.0,4.0,3.0,2014.0,9.0,0.444444


In [6]:
albums = pd.read_csv('albums.csv')

In [7]:
albums.head()

,band,name,kind,published,n_reviews,r_percent
0,Wardruna,Runaljod - gap var Ginnunga,Full-length,2009,6.0,98.0
1,Wardruna,The Pagan Herald No. III,Split,2009,NaN,NaN
2,Wardruna,Fehu,Single,2013,NaN,NaN
3,Wardruna,Runaljod - Yggdrasil,Full-length,2013,4.0,92.0
4,Wardruna,Loyndomsriss,Single,2014,NaN,NaN


In [63]:
# merging the dataframes:
both = pd.merge(left=bands, right=albums, left_on='name', right_on='band', how='left')

In [64]:
both.drop(['band', 'n_reviews', 'r_percent'], axis=1, inplace=True)

In [65]:
both.rename({'name_x':'band', 'name_y':'album'}, inplace=True, axis=1)

In [66]:
both.head()

,band,country,albums,singles_ep,splits,formed,age,productivity,album,kind,published
0,Wardruna,Norway,5.0,3.0,1.0,2003.0,20.0,0.25,Runaljod - gap var Ginnunga,Full-length,2009.0
1,Wardruna,Norway,5.0,3.0,1.0,2003.0,20.0,0.25,The Pagan Herald No. III,Split,2009.0
2,Wardruna,Norway,5.0,3.0,1.0,2003.0,20.0,0.25,Fehu,Single,2013.0
3,Wardruna,Norway,5.0,3.0,1.0,2003.0,20.0,0.25,Runaljod - Yggdrasil,Full-length,2013.0
4,Wardruna,Norway,5.0,3.0,1.0,2003.0,20.0,0.25,Loyndomsriss,Single,2014.0


In [57]:
# remove rows with missing values
both = both.dropna()

# Visualization with [altair](https://altair-viz.github.io/index.html)
- has to be installed!
- imported as `import altair as alt`
- quite similar in chaining operations to ggplot
- the base 'recipe' is as follows:
  - `alt.Chart(data)`-> data to be used
  - `.mark()` -> chart style (e.g. `mark_bar()`, `mark_line()` etc.)
  - `.encode(x=..., y=..., options)` -> what to encode from the data

In [59]:
!pip install altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import altair as alt

In [80]:
# simple scatter plot
alt.Chart(bands).mark_point().encode(
    x='formed',
    y='productivity',
)
# axis normally start at zero

alt.Chart(...)

In [79]:
# restrict the x axis to a specific domain
alt.Chart(bands).mark_point().encode(
    x=alt.X(
        'formed',
        scale=alt.Scale(domain=[1990, 2016])
        ),
    y='productivity',
)

alt.Chart(...)

In [136]:
# the cumulative count is a bit more involved than with ggplot2!

In [130]:
ot = both.groupby(['band', 'published']).count().reset_index()[['band', 'published', 'albums']]

In [131]:
csums=both.groupby(['band', 'published']).count().reset_index().groupby('band').cumsum()['albums']

In [132]:
ot['csums'] = csums

In [134]:
ot.head()

,band,published,albums,csums
0,Behemoth,1995.0,2,2
1,Behemoth,1996.0,1,3
2,Behemoth,1997.0,2,5
3,Behemoth,1998.0,1,6
4,Behemoth,1999.0,1,7


In [135]:
alt.Chart(
    ot
).mark_line().encode(
    x= alt.X('published', scale=alt.Scale(domain=[1990, 2023])),
    y='csums',
    color='band'
)

alt.Chart(...)